In [208]:
import carto2gpd
import cenpy
import dask.dataframe as dd
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import missingno as msno

import hvplot.pandas
import holoviews as hv

from io import BytesIO
import gzip
from urllib.request import urlopen

In [2]:
states = ['pa','il','ga','ca']
years = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']

# Part 1: Cluster Analysis

In [139]:
url = "https://lehd.ces.census.gov/data/lodes/LODES7/pa/wac/pa_wac_S000_JT00_"
wac = dd.concat([pd.read_csv(gzip.open(BytesIO(urlopen(url+year+".csv.gz").read()))).assign(year=year) for year in years])
wac['w_geocode'] = wac['w_geocode'].astype('str').str[:12]
wac.compute()

In [153]:
phl_bg = gpd.read_file("bg/tl_2019_42_bg/tl_2019_42_bg.shp").query("COUNTYFP in ['017','029','045','091','101']").to_crs('EPSG:3857')

In [180]:
wac_geo = phl_bg[['GEOID','geometry']].merge(wac.compute(), left_on='GEOID', right_on='w_geocode', how='inner')
wac_all = wac_geo.groupby('GEOID')['C000'].sum().reset_index()

In [181]:
wac_all.hvplot(geo=True,
                c="C000",
                crs=3857,
                hover_fill_color="gray",
                width=600, 
                height=400, 
                cmap='viridis')

:Curve   [index]   (C000)